# Seminararbeit: Machine Learned Hyperelastic Potentials
Moritz Trieu [3532051]

### The issue at hand
Every material (in this case hyperelastic materials) has an potential energy function. This energy functions is affected by the strain tensor and therefor lends itself to approximate the real-world energy function via deep learning. The strain tensor, as a symmetrical tensor, has 6 dregrees of freedom. But of course every degree is prone to measuring inaccuracies, so it might be that 2 strain tensors from the same material have slightly different values and are then approximated with differen values. 
A higher discrepancies of values might occur when measuring the same materiel from 2 different angles and/ or perspectives. In that case we would two rotated tensor, that mathematicially describe the same phenomenon from a different point of view. But as a neural network is unware of this fact, it might also approximate these two tensors with different values. 
To avoid this problem, we look to train the network with the tensor invariants - which are invariant under transformation - and/ or the eigenvalues of the tensor, which describe the main axis of strain of a given material. This in theory could give us a much better approximation as we are less prone to inaccuracies of the measured tensor.

### Open Questions
- Do we need a different model for every material?

## Initialize File

### About the conda environment
Python version: 3.10.4

Libraries:
- Pytorch
- Matplotlib
- Time

Import all neccassary libraries

In [21]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from time import perf_counter

Check for GPU avalibility and set global device status

In [4]:
system_device =  "cpu"

if torch.cuda.is_available() :
    print(f"CUDA is available on this system, setting device to GPU")
    system_device = "cuda"
else:
    print(f"CUDA not available on this system, setting device to CPU")

device = torch.device(system_device)

CUDA is available on this system, setting device to GPU


## Setting up Data

## Constructing General Model

In this section I create the overall template for every neural network I will be using in this project. As I want to compare the impact of different types inputs (for now the difference between the strain tensor and the strain tensor invariants, later on maybe the eigen-values of the strain tensor and any combination of those) the only difference between the networks should be the input size. The rest of the architecture stays the same and is a baseline architechture of 3 layers with a Leaky RelU activation function. This seems to be the best option for now as we aren't computing propabilities and Leaky RelU also combats the dead neuron issue.

## Comparison of different activation functions
Originally I wanted to use LeakyRelU as the default choice for activation functions. But as we need the derivation of the network function, the activation function needs to be differentiable. The RelU function is not differentiable, so there is the need to look into alternatives. 
### GelU
Gaussian Error Linear Unit is defined as GELU(x) = x * \Psi(x) where \Psi(x) is the standard Gaussian cumulative distribution function. The function can be approximated with GELU(x) = 0.5 * x * (1 + tanh(sqrt(2/\pi) * (0.044715 x ** 3))) or x * \sigma(1.702 * x)
Implemented in PyTorch. As it can be seen as a smoother RelU function, it is reasonable to assume that the function has similar shortcomings to RelU, mainly the dead neuron issue, even if not as pronounced. If we only work with smaller values between -2 and 2, this might not be a problem.

Sources: 
- https://paperswithcode.com/method/gelu
- https://arxiv.org/abs/1606.08415v4
- https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-27_at_12.48.44_PM.png

### Softplus

In [8]:
# General Hyperparameteres
hidden_size = 8
num_epochs = 100
batch_size = 100

class GeneralNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size=hidden_size):
        super(GeneralNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.l1 = nn.Linear(self.input_size, self.hidden_size)
        self.l2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.l3 = nn.Linear(self.hidden_size, 1)
        self.leaky_relu = nn.LeakyReLU()

    def forward(self, input):
        out = self.leaky_relu(self.l1(input))
        out = self.leaky_relu(self.l2(out))
        out = self.l3(out)

        return out


## Setting up STRAIN model
This is our baseline model with 6 degrees of freedom. The inputs are the values of the strain tensor.

In [9]:
strain_input_size = 6

strain_model = GeneralNeuralNet(strain_input_size).to(device)

## Setting up INVARIANT model
This is the model trained with the invariants of the tensors. The invariants are calculated with the measured tensors.

In [10]:
invariant_input_size = 3

invariant_model = GeneralNeuralNet(invariant_input_size)

## Loss, Optimizer, Scheduler
Note: I'm generating 2 different Loss functions here. As far as I understand, one would be sufficient. In every training loop the criterion is passed a torch.tensor with the predicted value and the tensor contains the grad history of the network, the training loop of every network should get it's own computation graph by autograd, regardless of which criterion is used or was used before. I just don't want to take any risks and have potentials bugs because of this. So having 2 different criterions is just a safety measure and might get replaced later if I see, that it will make no difference.

In [19]:
# Hyperparameters
learning_rate = 0.01

# Loss Function
strain_criterion = torch.nn.MSELoss() # TODO: Using MSE Loss for now, if the time is there implement a custom loss function
invariant_criterion = torch.nn.MSELoss()

#Optimizer
strain_optim = torch.optim.SGD(strain_model.parameters(), lr=learning_rate)
invariant_optim = torch.optim.SGD(invariant_model.parameters(), lr=learning_rate)

#Scheduler
strain_scheduler = torch.optim.lr_scheduler.StepLR(strain_optim, step_size=30, gamma=0.1)
invariant_scheduler = torch.optim.lr_scheduler.StepLR(invariant_optim, step_size=30, gamma=0.1)

## Train strain model

In [ ]:
# Save start time for training loop
strain_model_start_time = perf_counter()

for epoch in len(num_epochs):
    # Empty out grad
    strain_optim.zero_grad()

    # Convert Data

    # Forward Pass
    prediction = strain_optim(data) # TODO: Ingest actual data
    loss = strain_criterion(prediction, actual_values) # TODO: Ingest actual data

    # Backward Pass
    loss.backward()
    strain_optim.step()

    # Learning Rate Step
    strain_scheduler.step()

    if (epoch+1) % 10 == 0:
        print(f"STRAIN: Epoch: {epoch+1}/{num_epochs}: Loss: {loss.item():.4f} Learning Rate: {strain_scheduler.get_last_lr()} Time Passed: {(perf_counter() - strain_model_start_time):.4f}")

strain_model_end_time = perf_counter()
strain_model_time_delta = strain_model_end_time - strain_model_start_time

## Train Invariant Model

In [ ]:
# Save start time for training loop
invariant_model_start_time = perf_counter()

for epoch in len(num_epochs):
    # Empty out grad
    invariant_optim.zero_grad()

    # Convert Data

    # Forward Pass
    prediction = invariant_optim(data) # TODO: Ingest actual data
    loss = invariant_criterion(prediction, actual_values) # TODO: Ingest actual data

    # Backward Pass
    loss.backward()
    invariant_optim.step()

    # Learning Rate Step
    invariant_scheduler.step()

    if (epoch+1) % 10 == 0:
        print(f"INVARIANT Epoch: {epoch+1}/{num_epochs}: Loss: {loss.item():.4f} Learning Rate: {strain_scheduler.get_last_lr()} Time Passed: {(perf_counter() - strain_model_start_time):.4f}")

invariant_model_end_time = perf_counter()
invariant_model_time_delta = invariant_model_end_time - invariant_model_start_time

## Compute accuracies of different models

In [ ]:
with torch.no_grad():
    pass


## Compare Results of different models

In [ ]:
with torch.no_grad():
    pass


## Conclusion